In [2]:
from ortools.linear_solver import pywraplp
import pandas as pd

In [3]:
tasks = pd.read_csv('../data/tasks.csv')
crew = pd.read_csv('../data/crew.csv')

In [4]:
solver = pywraplp.Solver.CreateSolver('SCIP')


In [6]:
assignments = {}
for _, task in tasks.iterrows():
    for _, crew_member in crew.iterrows():
        assignments[(task['Task_ID'], crew_member['Crew_ID'])] = solver.BoolVar(
            f"assign_t{task['Task_ID']}_c{crew_member['Crew_ID']}")

In [7]:
objective = solver.Objective()
for (task_id, crew_id), var in assignments.items():
    objective.SetCoefficient(var, -1)  # Minimize idle time (negative coefficient)
objective.SetMinimization()


In [10]:
for _, task in tasks.iterrows():
    solver.Add(
        sum(assignments[(task['Task_ID'], crew_member['Crew_ID'])]
            for _, crew_member in crew.iterrows()) == 1)


In [11]:
status = solver.Solve()

In [13]:
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    results = []
    for (task_id, crew_id), var in assignments.items():
        if var.solution_value() > 0:
            results.append({"Task ID": task_id, "Crew ID": crew_id})
    results_df = pd.DataFrame(results)
    results_df.to_csv('../data/Optimal_Schedule.csv', index=False)
else:
    print("No optimal solution found.")

Optimal solution found!
